## Regular Transportation Problem

In [40]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets
nSources = 3
nDestinations = 4
I = range(1, nSources + 1)      # Sources
J = range(1, nDestinations + 1)    # Destinations

# Define variables
model.x = pyo.Var(I, J, domain=NonNegativeReals)

# Define cost matrix c[i][j]  
# change cost matrix as per # of sources and destinations.
c =  [[3, 2, 7, 6],
     [7, 5, 2, 3],
     [2, 5, 4, 5]]


# Objective function: Minimize total cost
model.obj = pyo.Objective(expr=sum(c[i-1][j-1]*model.x[i,j] for i in I for j in J), sense=minimize)

# Supply constraints (≤)
model.supply1 = pyo.Constraint(expr=sum(model.x[1,j] for j in J) <= 5000)
model.supply2 = pyo.Constraint(expr=sum(model.x[2,j] for j in J) <= 6000)
model.supply3 = pyo.Constraint(expr=sum(model.x[3,j] for j in J) <= 2500)

# Demand constraints (=)
model.demand1 = pyo.Constraint(expr=sum(model.x[i,1] for i in I) == 6000)
model.demand2 = pyo.Constraint(expr=sum(model.x[i,2] for i in I) == 4000)
model.demand3 = pyo.Constraint(expr=sum(model.x[i,3] for i in I) == 2000)
model.demand4 = pyo.Constraint(expr=sum(model.x[i,4] for i in I) == 1500)

# Solve
solver = pyo.SolverFactory('glpk')  
solver.solve(model)

# Print results
print("Optimal solution:")
for i in I:
    for j in J:
        print(f"x[{i},{j}] = {model.x[i,j].value}")

print(f"Total Cost = {model.obj.expr():.2f}")


Optimal solution:
x[1,1] = 3500.0
x[1,2] = 1500.0
x[1,3] = 0.0
x[1,4] = 0.0
x[2,1] = 0.0
x[2,2] = 2500.0
x[2,3] = 2000.0
x[2,4] = 1500.0
x[3,1] = 2500.0
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 0.0
Total Cost = 39500.00


## Capacitated Transportation Problems

In [42]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets
nSources = 3
nDestinations = 4
I = range(1, nSources + 1)      # Sources
J = range(1, nDestinations + 1)    # Destinations

# Define variables
model.x = pyo.Var(I, J, domain=NonNegativeReals)

# Define cost matrix c[i][j]  
# change cost matrix as per # of sources and destinations.
c =  [[3, 2, 7, 6],
     [7, 5, 2, 3],
     [2, 5, 4, 5]]


# Objective function: Minimize total cost
model.obj = pyo.Objective(expr=sum(c[i-1][j-1]*model.x[i,j] for i in I for j in J), sense=minimize)

# Supply constraints (≤)
model.supply1 = pyo.Constraint(expr=sum(model.x[1,j] for j in J) <= 5000)
model.supply2 = pyo.Constraint(expr=sum(model.x[2,j] for j in J) <= 6000)
model.supply3 = pyo.Constraint(expr=sum(model.x[3,j] for j in J) <= 2500)

# Demand constraints (=)
model.demand1 = pyo.Constraint(expr=sum(model.x[i,1] for i in I) == 6000)
model.demand2 = pyo.Constraint(expr=sum(model.x[i,2] for i in I) == 4000)
model.demand3 = pyo.Constraint(expr=sum(model.x[i,3] for i in I) == 2000)
model.demand4 = pyo.Constraint(expr=sum(model.x[i,4] for i in I) == 1500)

# Capacity constraints (=)
model.capacity1 = pyo.Constraint(expr=model.x[1,1] <= 2500)
model.capacity2 = pyo.Constraint(expr=model.x[3,3] >= 500)

# Solve
solver = pyo.SolverFactory('glpk')  
solver.solve(model)

# Print results
print("Optimal solution:")
for i in I:
    for j in J:
        print(f"x[{i},{j}] = {model.x[i,j].value}")

print(f"Total Cost = {model.obj.expr():.2f}")


Optimal solution:
x[1,1] = 2500.0
x[1,2] = 2500.0
x[1,3] = 0.0
x[1,4] = 0.0
x[2,1] = 1500.0
x[2,2] = 1500.0
x[2,3] = 1500.0
x[2,4] = 1500.0
x[3,1] = 2000.0
x[3,2] = 0.0
x[3,3] = 500.0
x[3,4] = 0.0
Total Cost = 44000.00


## General Transportation Model

In [29]:
import pandas as pd
import pyomo.environ as pyo

# ======= Read Excel Data =======
df = pd.read_excel('Data/Transportation Problem Data.xlsx')

# ======= Extract data =======
num_sources = df.shape[0] - 1
num_destinations = df.shape[1] - 2  # exclude first column and 'Capacity'

# Extract supply and demand
supply = df.iloc[:num_sources, -1].astype(int).tolist()
demand = df.iloc[-1, 1:-1].astype(int).tolist()

# Cost matrix
costs = df.iloc[:num_sources, 1:-1].astype(int).values.tolist()

# ======= Create Model =======
model = pyo.ConcreteModel()

# Sets (starting from 1 to match your formulation)
model.I = pyo.RangeSet(1, num_sources)         # Sources
model.J = pyo.RangeSet(1, num_destinations)    # Destinations

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers)

# Objective: Minimize total transportation cost
def obj_rule(model):
    return sum(costs[i-1][j-1] * model.x[i, j] for i in model.I for j in model.J)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Supply Constraints (≤)
def supply_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= supply[i - 1]

model.supply_con = pyo.Constraint(model.I, rule=supply_rule)

# Demand Constraints (=)
def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == demand[j - 1]

model.demand_con = pyo.Constraint(model.J, rule=demand_rule)

# ======= Solve =======
solver = pyo.SolverFactory('glpk')  
result = solver.solve(model, tee=True)

# ======= Print Results =======
# Output the values
print("\n\nNumber of Sources:", num_sources)
print("Number of Destinations:", num_destinations)
print("Supply:", supply)
print("Demand:", demand)
print("Cost Matrix:")
for row in costs:
    print(row)


print("\nTotal Cost =", pyo.value(model.obj))
for i in model.I:
    for j in model.J:
        val = pyo.value(model.x[i, j])
        if val > 0:
            print(f"Ship {val} units from Source {i} to Destination {j}")


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\laluv\AppData\Local\Temp\tmpuuvwpb5v.glpk.raw --wglp C:\Users\laluv\AppData\Local\Temp\tmpszhcj5sl.glpk.glp
 --cpxlp C:\Users\laluv\AppData\Local\Temp\tmpv03nehrp.pyomo.lp
Reading problem data from 'C:\Users\laluv\AppData\Local\Temp\tmpv03nehrp.pyomo.lp'...
7 rows, 12 columns, 24 non-zeros
12 integer variables, none of which are binary
91 lines were read
Writing problem data to 'C:\Users\laluv\AppData\Local\Temp\tmpszhcj5sl.glpk.glp'...
77 lines were written
GLPK Integer Optimizer, v4.65
7 rows, 12 columns, 24 non-zeros
12 integer variables, none of which are binary
Preprocessing...
7 rows, 12 columns, 24 non-zeros
12 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
7 ro